<a href="https://colab.research.google.com/github/rderakhshan/Falcon7b_text_gen_LLM_-_Langchain_test/blob/main/Falcon7b_text_gen_LLM_%26_Langchain_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Testing Falcon 7b instruct LLM** on GPU

**Note:** Switch runtime type to GPU before continuing

Falcon-40B-Instruct has already been finetuned on the conversational data. Though we'll work with the 7b, less space on colab.

We'll use the Transformers library to work with this model



**Install requirements & dependecies**

In [1]:
!pip install -q -U bitsandbytes
#!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
#!pip install -q -U git+https://github.com/huggingface/accelerate.git
#!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
#!pip install -q -U xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/

In [2]:
#install -transformers, accelerate, einops, and xformers
!pip install transformers accelerate einops xformers
#i can use either the git repo or direct pip install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 940.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 5.8 MB/s eta 0:00:00


**Bitsandbytes Configs**

Required for this project, initially tried falcon 7b without this but face ram issues due to model size.

This saves more memory at no additional performance

References: https://huggingface.co/blog/4bit-transformers-bitsandbytes

https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu

In [3]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    llm_int8_enable_fp32_cpu_offload=True
)

## **First Project: With falcon 7b intsruct directly**

**Goal:** To a **conversational chatbot** using Falcon 7b instruct model

In [4]:
#we'll use the Transformers library to work with this model

#install transformers accelerate einops xformers

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import transformers
import torch

#model_name = "tiiuae/falcon-7b-instruct"
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"
#https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded

In [5]:
#using the transformers library
#download tokeniser from the autotokenizer class
#download the model from the class AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             #device_map = device_map,
        quantization_config=quantization_config, trust_remote_code=True)


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [6]:
#use case here- text gen
#parameters
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    #torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    #device_map=device_map,
    max_length=200,
    use_cache=True,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)



In [7]:
#test on colab due to storage space on local pc
#test the llm-falcon7b by providing it with a query
sequences = pipeline(
    "Create a list of four important things a pilot should take note of"
)

#for seq in sequences:
#    print(f"Result: {seq['generated_text']}")

In [ ]:
sequences

[{'generated_text': "Create a list of four important things a pilot should take note of.\n1. Aircraft altitude and airspeed. 2. Aircraft attitude and heading. 3. Meteorological information, including weather and air pressure. 4. A review of the aircraft's maintenance logs and recent inspections."}]

In [8]:
#prompt 2
sequences = pipeline(
    "List five early signs of pregnancy in women"
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: List five early signs of pregnancy in women
- Missed menstrual periods or lighter periods
- Breast tenderness
- Fatigue or weakness
- Nausea or a sense of morning sickness
- Frequent urination


In [9]:
sequences #promt example 2 (different method)

[{'generated_text': 'List five early signs of pregnancy in women\n- Missed menstrual periods or lighter periods\n- Breast tenderness\n- Fatigue or weakness\n- Nausea or a sense of morning sickness\n- Frequent urination'}]

In [10]:
#prompt 3
sequences = pipeline(
    "List 6 ways of manufacturing a car for me, like a five year old. You can start with an intro of cars"
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: List 6 ways of manufacturing a car for me, like a five year old. You can start with an intro of cars.
1. Assemble parts on a workbench.
2. Use screws and bolts to connect them.
3. Insert wires and cable into slots.
4. Apply glue to hold them in place.
5. Put pieces together like a puzzle.
6. Use a power drill to attach them.


In [11]:
sequences #promt example 3 (different method)

[{'generated_text': 'List 6 ways of manufacturing a car for me, like a five year old. You can start with an intro of cars.\n1. Assemble parts on a workbench.\n2. Use screws and bolts to connect them.\n3. Insert wires and cable into slots.\n4. Apply glue to hold them in place.\n5. Put pieces together like a puzzle.\n6. Use a power drill to attach them.'}]

# **Falcon 7b Instruct with Langchain**

Creating a question answering app using LangChain & Falcon 7b instruct

In [12]:
# Some error in colab. fix with
#might be needful after running the above scripts to run langchain
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [13]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00


In [14]:
#Falcon LLM with LangChain
#LangChain has a pipeline called HuggingFacePipeline for models hosted in HuggingFace.

#install langchain package
#!pip install langchain

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import transformers
import torch

#model_name = "tiiuae/falcon-7b-instruct"
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"
#https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded -reference


#using the transformers library
#download tokeniser from the autotokenizer class
#download the model from the class AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             #device_map = device_map,
        quantization_config=quantization_config, trust_remote_code=True)

#use case here- text gen
#parameters
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    #torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    #device_map=device_map,
    max_length=200,
    use_cache=True,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [15]:
#create a pipeline for the Falcon model
from langchain import HuggingFacePipeline

#call the huggingface pipeline() object & pass the pipeline and model parameters
#temperature of 0, makes the model not to hallucinate much (make its own answers)
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

#Langchain contains PromptTemplate - allows to alter answers from the llm
#LLMChain - chains the prompttemplate and LLM together

from langchain import PromptTemplate, LLMChain



**Define Template -conversational 1**

In [16]:
#Define Template
#conversational

template = """
You are an intelligent chatbot that can function as a brand copywriter, customer service manager,
and have the ability to insert opinion on current affairs, media, trends, and general social commentary
when prompted. You will understand specific humor based off pop culture and media, sarcasm,
and social references.
Question: {query}
Answer:"""
prompt = PromptTemplate(template=template, input_variables=["query"])

#chain
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [17]:
query = "How do i pay for a service at the market? Write me an approach for this"

print(llm_chain.run(query))


You can pay for a service at the market in a variety of ways. One of the most common ways is by
using a credit/debit card to make payment online. You can also pay for services in cash or by using
mobile payment options. For example, if the service is offered by a mobile app, you can pay for it
using your phone's payment system. Additionally, some stores may accept payment by phone
through services like PayPal or Venmo. Finally, some services may offer payment through PayPal for
their customers who have an


In [35]:
query = "How Banks control the risk of banckrupcy in high interest rate environment?"

print(llm_chain.run(query))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Banks manage the risk of bankruptcy by diversifying their lending practices, maintaining
capital reserves and liquidity, and avoiding high risk investments. In addition, they have to comply
with the laws of their particular country or region. They also have to manage their risks by maintaining
a strong balance sheet, keeping loans in line with regulations, and by having a strong risk
management strategy. They must ensure they have enough capital and collateral to manage
their portfolio. Additionally, they must be able to quickly adapt to changing financial markets.
They must maintain adequate cash reserves in order to


In [36]:
query = "How AI can be benefitial in Banking's risk management area?"

print(llm_chain.run(query))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 AI can be beneficial for banks in a variety of ways. For example, AI can be used to
identify patterns in customer behavior, allowing banks to better assess and predict risk
associated with those customers. Additionally, AI can be used to automate and streamline
routine tasks, reducing manual labor and allowing employees more time to focus on
interacting with clients. AI can also improve compliance and security by flagging any
potentially suspicious activity and alerting the bank to investigate further. Ultimately,
AI can be used to improve decision-making by providing banks with more accurate


In [39]:
query = "write down the main tasks of Basel committee in banking industry in 10 bullet points?"

print(llm_chain.run(query))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



- Develop and enforce standards for banking and securities regulation
- Develop and enforce standards for banking and securities
- Supervision and examination of all banks
- Supervision of the securities industry and trading activities
- Financial innovation and emerging payment systems
- Basel capital requirements and capital adequacy
- International harmonization of accounting and auditing practices
- Development of risk measurement and risk management methods
- Development of international financial reporting standards
- Supervision and regulation of financial market infrastructures
- Financial consumer protection
- Financial education and public information
- Financial crime prevention



In [40]:
query = "How do i pump water into a tank from a bore hole ?"

print(llm_chain.run(query))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 You can pump water into a tank from a bore hole by using a well pump. The water is drawn up from the ground, and can then be transferred to the tank via pipelines. Alternatively, you can use surface-mounted pumps, which require an existing power supply and can be used to pump water from a bore hole, or from a river.
User 


In [41]:
query = "where can i request a refund of my money from an item i purchased previously but didnt meet my expectations ?"

print(llm_chain.run(query))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 you can usually request a refund or exchange from the retailer or seller you purchased the item from, depending on their store policy. You can also try to contact the item's manufacturer or distributor to see if they can offer any solutions or warranties. If the problem cannot be resolved through these options, you may also consider filing a complaint or dispute with the retailer or manufacturer, or seeking legal help if necessary.


In [42]:
query = "what Richard Philips Feynman did to get the nobel prize in Physics?"

print(llm_chain.run(query))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Richard Feynman received the Nobel Prize in Physics in 1962 for his work on the Theory of Quantum Electrodynamics and his pioneering research in nuclear physics. He was known for his creative, intuitive and innovative methods, and was also known for his ability to explain complex topics using simple language and analogies. He would often draw visual aids to illustrate his points and would engage in playful banter with his audience.
User 


In [49]:
query = "Write down in three paragraph, bibliography of Richard Philips Feynman."

print(llm_chain.run(query))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Richard Feynman, a renowned physicist, was born in 1916 in El Paso, Texas. He graduated from Princeton
in 1938 where he studied theoretical physics. Throughout his career, Feynman worked in various fields of
physics, including the quantum theory, nuclear physics, and theoretical mechanics. He is most well known
for his work in developing the theory of quantum mechanics. 

In his later years, Feynman's work in quantum mechanics led him to make important contributions to the
development of modern physics. In 1962,


In [50]:
query = "Write down a steps on the steps which one should take to establish good habits."

print(llm_chain.run(query))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



1. Establish a clear vision for why you need to establish good habits.
2. Develop an actionable plan to create the habits you want to achieve.
3. Set a timeline of how long it will take to create the habits.
4. Establish a tracking method that will allow you to measure your progress.
5. Make a habit chart and place it on your workspace or in your home.
6. Establish an accountability system. Choose someone or yourself to check-in with regularly.
7. Celebrate your success and don't give up if


**Alterative template** - Question answering

In [52]:
query = "Write down a cmprehensive list of unique features of Dutch language."

print(llm_chain.run(query))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



- The Dutch language has 25 different dialects and accents
- Dutch is the most popular language in Europe
- The Dutch language has over 30,000 words, compared to English which has over 50,000 words
- Dutch grammar has 4,000 different rules compared to English
- Dutch has the most amount of different words for colors, more than any other language in the world
- Dutch has more than 1000 regional dialects, while English has just 12 regional dialects
- Dutch has 5 different cases for nouns and verbs



In [53]:
query = "Write an email to head of treasury department to ask his/her availibilitie to conduct a meeting."

print(llm_chain.run(query))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Subject: Request for Treasury Department Head's Availability 
Dear [Treasury Department Head's Name],

I am reaching out to request your availability for a meeting with our team. As you may know,
our team has been discussing some important matters related to our organization, and we believe that
your expertise could be extremely beneficial in helping us make an important decision.

Our team is available on [date(s)], and we would be happy to schedule our meeting at your convenience.

Thank you


In [54]:
query = "Write an email to head of treasury department to ask for a meeting to agre on proposed roasmap."

print(llm_chain.run(query))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Subject: Requesting Meeting to Discuss Proposed Revenue Strategy

Dear [Treasury Manager],

I am writing to request a meeting with you to discuss our proposed revenue strategy. Our team has
been working hard to develop a comprehensive plan in order to increase our company's revenue. I
believe that by sitting down and discussing our ideas could be beneficial for both parties.

Would it be possible to schedule a time that we could meet? I understand that your schedule may be
tight, but I think it would be a


In [62]:
query = "Write an email to head of treasury department. First appologize on delay. Then, explain reasons of delay."

print(llm_chain.run(query))

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



Dear Sir,

I am writing to apologize for the delay in submitting my quarterly report. I understand that
timeliness is of utmost importance, and I have failed to meet your expectations. There were several
challenges that arose, and I want to assure you that I am taking appropriate actions to avoid such
delays in future.

The reasons for the delay were:

1. Inefficient software used in preparing the financial reports.
2. Unexpected increase in workload due to new business development.



In [20]:
template1 = """Question: {question}
Answer: Let's think step by step."""

prompt1 = PromptTemplate(
    template=template1,
    input_variables= ["question"]
)

#chain
llm_chain1 = LLMChain(prompt=prompt1, llm=llm)

**More Tests**

In [21]:
llm_chain("How to prepare nigerian eba?")

{'query': 'How to prepare nigerian eba?',
 'text': '\n1. Wash hands.\n2. Wash the eba leaves.\n3. Soak the eba leaves in hot water.\n4. Drain off excess water from the soaked eba leaves.\n5. Boil water.\n6. Strain the boiled water through the soaked and drained eba leaves.\n7. Serve eba and other side dishes.\nAnswer: 7'}

In [22]:
llm_chain("How to start an aircraft?")

{'query': 'How to start an aircraft?', 'text': ' You need a push.\nUser '}

**More Templates** Question answering/Summarisation

In [23]:
template2 = """Question: /n {question}. Answer: """

prompt2 = PromptTemplate(
    template=template2,
    input_variables= ["question"]
)


llm_chain_2 = LLMChain(prompt=prompt2, llm=llm)

In [24]:
result_explanation = llm_chain_2("Explain Fungi in detail to me")

In [25]:
result_explanation['text']


'\nFungi are eukaryotic organisms that typically include simple, single-celled organisms or multicellular organisms. They are classified in the Kingdom Fungi, and are closely related to plants and animals. Fungi play important roles in many ecological ecosystems, and often act as decomposers, providing essential nutrients to other life forms. Fungi are capable of producing their own food through the process of fermentation or decomposition. Common types of fungi include mushrooms, yeasts, and molds. These organisms also contribute to the production of some foods and beverages, such as bread, beer, and wine.'

In [26]:
llm_chain_2("who is a customer service manager, explain like a five year old?")

{'question': 'who is a customer service manager, explain like a five year old?',
 'text': '/a customer is a nice person who buys things from my store.\nAs a customer service manager, I am here to help customers find what they need. When someone comes into my store, I try my best to make them feel like nice, happy customers. I work with the best products, so they can have the best experience possible. If someone needs help finding something or has a question, I try to answer it as fast as I can. And if they are nice and polite, I will make sure to give them extra special treatment.'}

In [27]:
prompt_code = """ I have to pass 2 values that are as a string

'5,7'

And turn them into a list

(5,7), help with the the procedures?
"""


In [28]:
llm_chain_2(prompt_code)

{'question': " I have to pass 2 values that are as a string\n\n'5,7'\n\nAnd turn them into a list\n\n(5,7), help with the the procedures?\n",
 'text': '<code>(5,7), help with the procedures?</code>\n\nAs an AI language model, I cannot provide a specific solution for this problem. However, I can provide some general guidance on how to solve this problem:\n\n1. You can convert the string into a list using the following syntax in your preferred programming language:\n\n<code>\nlist(list("5", "7"), list("8", "9")),\n</code>\n\nHere, the first element is the string value you want to convert from the original string, and the second element is a separate list that will hold the elements that make up that string. You can then use this list as an argument in various programming functions or'}

In [29]:
llm_chain_2("How to select everything in a sql table, please explain too?")


{'question': 'How to select everything in a sql table, please explain too?',
 'text': '/n \nTo select everything in a SQL table, you can use the asterisk (*) notation. The following command would select all columns in the table:\n\n```sql\nSELECT * FROM [table];\n```'}

**Conversational**

In [30]:
prompt_sql = """
Table departments, columns = [DepartmentId, DepartmentName]
Table students, columns = [DepartmentId, StudentId, StudentName]
Create a Postgresql query for all students in the Computer Science Department
"""
llm_chain_2(prompt_sql)

{'question': '\nTable departments, columns = [DepartmentId, DepartmentName]\nTable students, columns = [DepartmentId, StudentId, StudentName]\nCreate a Postgresql query for all students in the Computer Science Department\n',
 'text': '/n\n<pre><code>SELECT "c".* FROM "c".table_name("c") c, "s".table_name("s") s\nWHERE s."s".table_name().value = "CSCD";\n</code></pre>'}

In [31]:
llm_chain_2("show me how python's args and kwargs work")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': "show me how python's args and kwargs work",
 'text': "/n\nPython's argparse module allows you to easily define and use command-line arguments in the same way that shell commands work.\n\nTo define arguments, you use sub-commands with the `subparser` object, then add them as `add_argument()` function to the `parser` object.\n\nTo define optional arguments, add the `default` argument to the `add_argument()` function with a default value.\n\nTo define optional arguments that can have multiple options, add multiple `add_arguments()` in the `parser` object.\n\nFor example, to define a command for the `argparser` module to parse the options for a `my_parser` object in the `my_parser` class, you can use the following code:\n\n```python\nmy_parser"}

In [32]:
llm_chain_2("Write me a business plan, my goal is to build a nike competitor")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': 'Write me a business plan, my goal is to build a nike competitor',
 'text': '/n The business plan should focus on product innovation, customer marketing, and strategic partnerships. To become a nike competitor, one should focus on building a high quality product with unique features and a well defined brand identity. This will enable the company to gain market share by providing a competitive pricing strategy. Additionally, the strategy should focus on developing unique marketing strategies, such as using celebrities and other creative media to create a unique brand identity. /n /n /n /n /n /n /n /n /n /n'}

In [33]:
llm_chain_2("what is Python's ABC library and what is it for?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': "what is Python's ABC library and what is it for?",
 'text': '/n\nPyTorch is a deep learning framework based on torch, the C programming language, and the Python programming language. It is used in applications such as object recognition, image classification, and text summarization using natural language processing. It has become popular among developers and researchers due to its fast processing capabilities and its support for Python developers. PyTorch also supports many programming languages like C++, Java, R, and C++.'}

In [34]:
llm_chain_2("what is carbon capture, is cloud hosting involved in it?")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'question': 'what is carbon capture, is cloud hosting involved in it?',
 'text': '/n Yes.\nCarbon capture is a process that involves the recovery of carbon dioxide and other greenhouse gas emissions from industrial processes. Cloud hosting is a service that allows organizations to access resources and services on a virtualized infrastructure. So, while cloud hosting is not directly involved in carbon capture, it is a technology that can be used as part of the solution.'}

**Observations:**

Falcon 7b is cool though not yet there and has limitations, it wasn't able to understand some unique nationalities terminologies (like in my case, Nigerian food). Falcon 40b would definitely be better.

It's opensource and commercially available.
